In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import string, re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carolineshi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
allreviews_df = pd.read_pickle("fulldf.pkl")

In [6]:
allreviews_df

,id,review,processed,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,genre,label,year_released
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",trip hop eventu becam punchlin press shorthand...,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,electronic,virgin,1998.0
1,22721,"Eight years, five albums, and two EPs in, the ...",eight year five album two ep new york base out...,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017,metal,hathenter,2016.0
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,minneapoli uranium club seem revel aggress obt...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,rock,static shock,2016.0
3,22661,Kleenex began with a crash. It transpired one ...,kleenex began crash transpir one night long fo...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,rock,kill rock stars,2016.0
4,22725,It is impossible to consider a given release b...,imposs consid given releas footwork artist wit...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017,electronic,teklife,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18396,1535,The marketing guys of yer average modern megac...,market guy yer averag modern megaconglomer lov...,let us replay!,coldcut,http://pitchfork.com/reviews/albums/1535-let-u...,8.9,0,james p. wisdom,None,1999-01-26,1,26,1,1999,jazz,ninja tune,NaN
18397,1341,"Well, it's been two weeks now, and I guess it'...",well two week guess time tell guy someth cassi...,1999,cassius,http://pitchfork.com/reviews/albums/1341-1999/,4.8,0,james p. wisdom,None,1999-01-26,1,26,1,1999,electronic,astralwerks,NaN
18398,5376,"Out of Tune is a Steve Martin album. Yes, I'l...",tune steve martin ye explain upon time steve m...,out of tune,mojave 3,http://pitchfork.com/reviews/albums/5376-out-o...,6.3,0,jason josephes,contributor,1999-01-12,1,12,1,1999,rock,4ad,NaN
18399,2413,"Well, kids, I just went back and re-read my re...",well kid went back read review guy last burn n...,"singles breaking up, vol. 1",don caballero,http://pitchfork.com/reviews/albums/2413-singl...,7.2,0,james p. wisdom,None,1999-01-12,1,12,1,1999,rock,touch and go,1999.0


In [7]:
rock_df = allreviews_df.loc[allreviews_df['genre'] == 'rock']

In [8]:
rock_df

,id,review,processed,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,genre,label,year_released
2,22659,Minneapolis’ Uranium Club seem to revel in bei...,minneapoli uranium club seem revel aggress obt...,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,rock,static shock,2016.0
3,22661,Kleenex began with a crash. It transpired one ...,kleenex began crash transpir one night long fo...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,rock,kill rock stars,2016.0
21,22718,Nine Inch Nails mastermind Trent Reznor has sp...,nine inch nail mastermind trent reznor spent d...,not the actual events ep,nine inch nails,http://pitchfork.com/reviews/albums/22718-not-...,6.3,0,benjamin scheim,contributor,2016-12-28,2,28,12,2016,rock,self-released,2016.0
23,22695,It’s been nearly fourteen years since we last ...,nearli fourteen year sinc last heard new belov...,"early tapes, 1996-1998",the microphones,http://pitchfork.com/reviews/albums/22695-earl...,6.5,0,benjamin scheim,contributor,2016-12-27,1,27,12,2016,rock,p.w. elverum & sun,2016.0
26,22532,2016 has been a surreal and absurd year for mo...,surreal absurd year whirlwind must melina dute...,turn into,jay som,http://pitchfork.com/reviews/albums/22532-turn...,8.2,0,quinn moreland,assistant editor,2016-12-23,4,23,12,2016,rock,polyvinyl,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18387,8835,The concept of late work isn't invoked in too ...,concept late work invok mani record review dou...,"apple venus, volume one",xtc,http://pitchfork.com/reviews/albums/8835-apple...,8.2,0,zach hooker,None,1999-02-23,1,23,2,1999,rock,tvt,NaN
18389,1242,Remember back in elementary school when multim...,rememb back elementari school multimedia educ ...,califone,califone,http://pitchfork.com/reviews/albums/1242-calif...,9.2,0,mark richard-san,None,1999-02-18,3,18,2,1999,rock,flydaddy,1999.0
18390,1023,The ripples from Jawbox's dive into the underg...,rippl jawbox dive underground rock pond still ...,mission: control!,burning airlines,http://pitchfork.com/reviews/albums/1023-missi...,8.4,0,brent dicrescenzo,contributor,1999-02-16,1,16,2,1999,rock,desoto,NaN
18398,5376,"Out of Tune is a Steve Martin album. Yes, I'l...",tune steve martin ye explain upon time steve m...,out of tune,mojave 3,http://pitchfork.com/reviews/albums/5376-out-o...,6.3,0,jason josephes,contributor,1999-01-12,1,12,1,1999,rock,4ad,NaN


## Rock Reviews EDA

## Count Vectorizer

In [9]:
cv = CountVectorizer()

In [10]:
words_cv = cv.fit_transform(rock_df['processed'])

In [11]:
cv_df = pd.DataFrame(words_cv.toarray(), columns = cv.get_feature_names()).head(20)

## TF-IDF Vectorizer

In [9]:
tfidf_vectorizer = TfidfVectorizer()
words_tfidf = tfidf_vectorizer.fit_transform(rock_df['processed'])

In [10]:
tfidf_df = pd.DataFrame(words_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names()).head(20)

In [11]:
tfidf_df

,aa,aaa,aaaa,aaaaa,aaaaaaaaa,aaaaaah,aaaaah,aaaaay,aaaahhhh,aaaahhhhhhkkkkghhh,...,zyklon,zyko,zynth,zyx,zz,zzz,zzzpenchant,zzzz,zzzzzzzz,zzzzzzzzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TF-IDF Vectorizer x NMF Topic Modeling

In [30]:
nmf_model = NMF(10)
doc_topic = nmf_model.fit_transform(words_tfidf)

In [31]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["topic_1", "topic_2", 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10'],
             columns = tfidf_vectorizer.get_feature_names())
topic_word

,aa,aaa,aaaa,aaaaa,aaaaaaaaa,aaaaaah,aaaaah,aaaaay,aaaahhhh,aaaahhhhhhkkkkghhh,...,zyklon,zyko,zynth,zyx,zz,zzz,zzzpenchant,zzzz,zzzzzzzz,zzzzzzzzzzzz
topic_1,0.003,0.002,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.001,...,0.0,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.0,0.001
topic_2,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000
topic_3,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.0,0.000
topic_4,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.0,0.000
topic_5,0.000,0.000,0.001,0.001,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000
topic_6,0.001,0.001,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.0,0.000
topic_7,0.001,0.000,0.000,0.000,0.0,0.0,0.001,0.000,0.0,0.001,...,0.0,0.000,0.002,0.000,0.012,0.000,0.000,0.001,0.0,0.000
topic_8,0.000,0.006,0.000,0.000,0.0,0.0,0.000,0.001,0.0,0.000,...,0.0,0.001,0.000,0.000,0.000,0.001,0.001,0.000,0.0,0.000
topic_9,0.001,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000
topic_10,0.000,0.001,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000


In [32]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [33]:
display_topics(nmf_model, tfidf_vectorizer.get_feature_names(), 15)


Topic  0
get, thing, go, one, good, know, say, time, make, guy, peopl, want, think, realli, even

Topic  1
sound, guitar, track, record, instrument, drum, vocal, minut, drone, nois, piec, work, one, feel, melodi

Topic  2
band, rock, punk, guitar, sound, pop, indi, post, group, debut, song, riff, vocal, choru, new

Topic  3
pollard, gbv, guid, sprout, bob, robert, tobia, voic, record, circu, spaceship, lane, suitcas, song, tobin

Topic  4
live, record, version, disc, releas, band, set, origin, album, cover, perform, song, year, one, fan

Topic  5
remix, danc, pop, synth, disco, track, hous, beat, electro, origin, ep, singl, new, dj, mix

Topic  6
black, metal, rock, blue, white, riff, heavi, doom, death, minut, mountain, cave, sabbath, nois, guitar

Topic  7
love, song, sing, voic, lyric, songwrit, pop, countri, life, folk, singer, young, emot, feel, self

Topic  8
xiu, stewart, jami, fabul, larsen, mcelroy, muscl, cover, fag, luv, patrol, shearwat, yoshii, emot, ep

Topic  9
smith, f

In [34]:
Topic_Matrix = nmf_model.transform(words_tfidf)

topic_matrix_df = pd.DataFrame(Topic_Matrix).add_prefix('topic_')
topic_matrix_df['DESCRIPTIONS'] = rock_df['review']
topic_matrix_df['ALBUM_SCORE'] = rock_df['score']
topic_matrix_df['GENRE'] = rock_df['genre']

topic_matrix_df.dropna(inplace=True)

In [35]:
topic_matrix_df

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,DESCRIPTIONS,ALBUM_SCORE,GENRE
2,0.010326,0.022722,0.017677,0.002974,0.021516,0.049280,0.011949,0.005156,0.000000,0.000000,Minneapolis’ Uranium Club seem to revel in bei...,7.3,rock
3,0.009052,0.024873,0.025844,0.004646,0.053143,0.000000,0.001577,0.006701,0.000000,0.000447,Kleenex began with a crash. It transpired one ...,9.0,rock
21,0.052566,0.000000,0.008027,0.000000,0.007818,0.007480,0.005488,0.020632,0.001293,0.003483,Nine Inch Nails mastermind Trent Reznor has sp...,6.3,rock
23,0.015341,0.000000,0.018406,0.000000,0.057299,0.000000,0.022485,0.007325,0.000000,0.000000,It’s been nearly fourteen years since we last ...,6.5,rock
26,0.001469,0.063849,0.013905,0.000000,0.007767,0.004867,0.003068,0.000000,0.001760,0.000000,2016 has been a surreal and absurd year for mo...,8.2,rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9431,0.056233,0.004043,0.025290,0.001469,0.019266,0.000685,0.011419,0.013940,0.000000,0.000000,"Nu-rave may be over, but Late of the Pier make...",6.7,rock
9433,0.066392,0.002984,0.000000,0.004167,0.044346,0.000000,0.000000,0.007404,0.000000,0.000000,"Over the past decade, there has been grumbling...",7.6,rock
9434,0.025098,0.035442,0.009653,0.002467,0.012716,0.000332,0.027036,0.001655,0.000000,0.004235,"""Safe for the Time Being"", the last song on Ma...",6.9,rock
9435,0.039313,0.003912,0.023705,0.000000,0.012979,0.000569,0.000000,0.000000,0.000000,0.000000,Any attempt to offer an authoritative overview...,7.3,rock


In [17]:
def Hot_Topic(df,topic,num):
    for Description in df.sort_values(by=topic, ascending=False).head(num)['DESCRIPTIONS'].values:
        print(Description)
        print()

def Hot_Words(df,topic,num):
    HW = df.sort_values(by=topic, ascending=False).head(num)
    return HW

In [22]:
Hot_Topic(topic_matrix_df, 'topic_5', 10)

Sébastien Tellier is still best known in the States as the louche, besunglassed Frenchman whose 2008 album Sexuality layered the electro-disco sound that was big in Paris at the time with thick dollops of a particularly retro kind of theatrical sleaziness distilled from early-VHS-era pornography. But while he certainly excels at the role of the Eurotrash slimeball, he actually has a far broader range than that. A devotee of Serge Gainsbourg, Tellier has inherited the master’s ambitious genre-hopping habit (as well as his arch-Gallic oleaginity), and aside from house-y synthpop he’s recorded Gainsbourg-style baroque pop, slightly skewed takes on contemporary radio pop, an album of piano ballads, and a film score.For his new Confection, Tellier returns to his role as a composer for film in spirit, if not in actual fact, and combines it with Sexuality’s pursuit of bygone erotic aesthetics. Long story short, it sounds like the soundtrack to a 70s French porn movie that, as far as I can tel

In [137]:
!pip install wordcloud

     |████████████████████████████████| 220 kB 3.7 MB/s eta 0:00:01
  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp38-cp38-macosx_10_9_x86_64.whl size=157999 sha256=ed1fafb15dfd91f0f95fd1aae8169ebcb96b6b4f25787ce0cb2140348faa0c68
  Stored in directory: /Users/carolineshi/Library/Caches/pip/wheels/4d/3f/0d/a2ba9b7895c9f1be89018b3141c3df3d4f9c786c882ccfbc3b
Successfully built wordcloud


## Count Vectorizer x LDA Topic Modeling

In [23]:
lda = LatentDirichletAllocation(n_components = 10, n_jobs = -1, random_state = 42)

In [24]:
lda_fit = lda.fit(words_cv)

In [25]:
display_topics(lda, cv.get_feature_names(), 15)


Topic  0
oberst, lewi, conor, eye, toth, martin, bright, weiss, rilo, kiley, mau, coldplay, hagan, one, breeder

Topic  1
band, sound, track, guitar, record, one, pop, song, vocal, make, rock, time, even, new, much

Topic  2
band, rock, record, song, one, pop, guitar, new, sound, track, even, two, time, make, dylan

Topic  3
band, record, rock, one, sound, song, track, time, even, punk, year, new, live, releas, guitar

Topic  4
sound, guitar, record, track, one, feel, song, make, voic, work, vocal, time, seem, way, instrument

Topic  5
sound, one, track, record, work, song, band, guitar, rock, time, two, first, new, even, year

Topic  6
cox, johnston, jone, black, deerhunt, buckner, higg, go, metal, eitzel, life, olsen, hain, meiburg, lungfish

Topic  7
band, song, sound, one, record, pop, love, even, make, track, time, year, new, first, rock

Topic  8
band, sound, one, record, song, rock, even, time, track, get, much, make, seem, pollard, year

Topic  9
rock, band, one, get, time, ma

In [26]:
topic_word = pd.DataFrame(lda.components_.round(3),
             index = ["topic_1", "topic_2", 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10'],
             columns = cv.get_feature_names())
topic_word

,aa,aaa,aaaa,aaaaa,aaaaaaaaa,aaaaaah,aaaaah,aaaaay,aaaahhhh,aaaahhhhhhkkkkghhh,...,zyklon,zyko,zynth,zyx,zz,zzz,zzzpenchant,zzzz,zzzzzzzz,zzzzzzzzzzzz
topic_1,0.100,0.100,0.100,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.100,0.1,0.1,0.1,0.1,0.1
topic_2,1.859,2.810,0.120,1.1,1.1,0.1,0.1,1.1,0.1,0.1,...,1.1,0.1,0.1,0.1,2.319,1.1,0.1,0.1,0.1,0.1
topic_3,1.005,0.100,0.100,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.100,0.1,0.1,1.1,0.1,0.1
topic_4,0.100,0.100,0.100,0.1,0.1,0.1,1.1,0.1,0.1,0.1,...,0.1,0.1,0.1,1.1,7.422,0.1,0.1,0.1,0.1,1.1
topic_5,3.076,0.100,0.100,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.100,0.1,0.1,2.1,0.1,0.1
topic_6,0.100,0.100,0.100,0.1,0.1,1.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.100,0.1,0.1,0.1,1.1,0.1
topic_7,1.608,0.127,0.100,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.100,0.1,1.1,0.1,0.1,0.1
topic_8,0.835,1.979,2.079,0.1,0.1,0.1,0.1,0.1,1.1,1.1,...,0.1,0.1,0.1,0.1,0.100,0.1,0.1,0.1,0.1,0.1
topic_9,0.100,7.909,0.100,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,1.1,4.1,0.1,2.048,0.1,0.1,1.1,0.1,0.1
topic_10,4.217,5.675,0.100,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,16.611,0.1,0.1,0.1,0.1,0.1


In [27]:
Topic_Matrix = lda.transform(words_cv)

topic_matrix_df = pd.DataFrame(Topic_Matrix).add_prefix('topic_')
topic_matrix_df['DESCRIPTIONS'] = rock_df['review']
topic_matrix_df['ALBUM_SCORE'] = rock_df['score']
topic_matrix_df['GENRE'] = rock_df['genre']

topic_matrix_df.dropna(inplace=True)

In [28]:
topic_matrix_df

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,DESCRIPTIONS,ALBUM_SCORE,GENRE
2,0.000217,0.690881,0.000217,0.027309,0.000217,0.000217,0.000217,0.000217,0.000217,0.280291,Minneapolis’ Uranium Club seem to revel in bei...,7.3,rock
3,0.000217,0.000780,0.000217,0.410161,0.158073,0.239586,0.000217,0.059093,0.082406,0.049250,Kleenex began with a crash. It transpired one ...,9.0,rock
21,0.000147,0.000147,0.000147,0.000147,0.000147,0.000147,0.000147,0.000147,0.132690,0.866132,Nine Inch Nails mastermind Trent Reznor has sp...,6.3,rock
23,0.000167,0.000167,0.000167,0.329838,0.000167,0.008828,0.000167,0.000167,0.000167,0.660162,It’s been nearly fourteen years since we last ...,6.5,rock
26,0.000380,0.236274,0.000380,0.000380,0.660591,0.000380,0.000380,0.000380,0.100473,0.000380,2016 has been a surreal and absurd year for mo...,8.2,rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9431,0.000215,0.330523,0.072057,0.000215,0.000215,0.000215,0.000215,0.066589,0.218145,0.311609,"Nu-rave may be over, but Late of the Pier make...",6.7,rock
9433,0.000319,0.396084,0.000319,0.000319,0.000319,0.000319,0.000319,0.366722,0.023536,0.211747,"Over the past decade, there has been grumbling...",7.6,rock
9434,0.000455,0.409705,0.000455,0.000455,0.000455,0.456526,0.000455,0.000455,0.000455,0.130586,"""Safe for the Time Being"", the last song on Ma...",6.9,rock
9435,0.266647,0.273941,0.000245,0.126199,0.000245,0.000245,0.000245,0.000661,0.000245,0.331325,Any attempt to offer an authoritative overview...,7.3,rock


## Running Random Forest with just TF-IDF/NMF topics as features

In [36]:
X = topic_matrix_df.iloc[:, 0:10]
y = topic_matrix_df.ALBUM_SCORE

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [38]:
from sklearn.ensemble import RandomForestRegressor
randomforest = RandomForestRegressor(n_estimators=100)
randomforest.fit(X_train, y_train)

RandomForestRegressor()

In [39]:
y_train_pred = randomforest.predict(X_train)

In [40]:
act_pred_df = pd.DataFrame({'Actual': y_train, 'Predicted': y_train_pred, 'Residual': abs(y_train-y_train_pred)})

In [41]:
act_pred_df

,Actual,Predicted,Residual
1577,8.0,7.477,0.523
640,7.4,7.160,0.240
1243,6.9,6.513,0.387
4401,6.4,6.666,0.266
9382,5.1,5.802,0.702
...,...,...,...
8085,5.1,5.522,0.422
1402,8.7,8.089,0.611
7464,4.9,5.739,0.839
8648,5.1,5.786,0.686


In [42]:
rf_test_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
rf_test_mae = mean_absolute_error(y_train, y_train_pred)

rf_test_r2 = randomforest.score(X_train, y_train)

print(rf_test_rmse)
print(rf_test_mae)
print(rf_test_r2)

0.46547199309974246
0.3526873406259056
0.8538708251236511


## Trying Corex!

In [1]:
!pip install corextopic
!pip install networkx

In [2]:
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

In [13]:
words = list(np.asarray(cv.get_feature_names()))

In [15]:
topic_model = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model.fit(words_cv, words=words, docs=rock_df.processed)

In [16]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: peopl,year,would,time,could,ever,even,made,thing,day
1: love,girl,got,boy,fuck,know,kid,man,self,want
2: drone,electron,nois,ambient,percuss,piec,textur,instrument,drum,minut
3: emot,sing,lyric,voic,life,person,piano,arrang,songwrit,relationship
4: reissu,version,origin,includ,releas,cover,live,perform,compil,disc
5: punk,riff,rock,band,metal,hardcor,drummer,scream,post,bassist
6: pop,disco,singl,synth,uk,indi,danc,electro,remix,radio
7: kawabata,makoto,jaki,via,homm,freak,sui,liebezeit,fu,generi
8: cultur,film,packag,art,soundtrack,featur,dvd,movi,non,upon
9: trux,segal,ty,royal,melvin,hagerti,jehu,dwyer,dashboard,brock


In [20]:
topic_model.get_top_docs(topic=8, n_docs=2)

[('probabl reason biograph strong pop tradit lou reed john cale advoc geniu andi warhol success song drella neutral milk hotel indi rock touchston aeroplan sea draw life stori ann frank unlik hollywood biopic strike oscar gold almost annual bio album far anyon popular bio concept gruff rhi bryan hollon whether solo frontman long run welsh psych pop anarchist super furri anim rhi alway knack commun lofti idea sli wit warm feel memor invent set hollon accomplish experiment hip hop produc dj right boom bip updat new wave electro master first rhi neon neon stainless style though mercuri prize nomin full length inspir life automak john delorean sens neon neon second effort repres first true bio silveri retro futurist excess greed good decad pervad stainless style lyric refer engin playboy life certainli present record stuf great song human retro whether knew stori attach praxi make perfect song never quit hit high predecessor best track coher still strang reward way imposs separ subject sub